In [1]:
%matplotlib inline
import scipy.io
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
from wide_resnet import WideResNet
from MyModel import MyModel
from keras.utils import to_categorical
from keras.models import Sequential




os.environ["CUDA_VISIBLE_DEVICES"]="1"
# load model and weights
img_size = 224
#model64 = MyModel(img_size)()
#model64.load_weights(os.path.join("models", "WRN_16_8.h5"))

#img_size32 = 32
#model32 = WideResNet(img_size32, depth=16, k=8)()
#model32.load_weights(os.path.join("../../dataset/checkpoints", "model32.hdf5"))


/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
db = "wiki"
data = scipy.io.loadmat(os.path.join("../../dataset/", "{}32.mat".format(db)))
test_images = data["image"]
test_genders = data["gender"][0]
test_ages = data["age"][0]

cols, rows = 4, 3
img_num = cols * rows
path_root = "data/{}_crop/".format(db)
img_ids = np.random.choice(len(test_ages), img_num)
sub_test_images = test_images[img_ids,:,:,:]

# predict
results = model32.predict(sub_test_images)
predicted_genders = results[0]
ages = np.arange(0, 101).reshape(101, 1)
predicted_ages = results[1].dot(ages).flatten()

for i in range(img_num):
    plt.subplot(rows, cols, i + 1)
    plt.imshow(cv2.cvtColor(sub_test_images[i,:,:,:], cv2.COLOR_BGR2RGB))
    plt.title("{}, {}".format(int(predicted_ages[i]),
                              "F" if predicted_genders[i][0]>0.5 else "M"))
    plt.axis('off')
plt.savefig("result.png")

In [ ]:
from os import listdir
from os.path import isfile, join
import face_recognition
mypath='test/'
onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
#images32 = np.empty(len(onlyfiles), dtype=object)
#faces32 = np.empty((len(onlyfiles), img_size32, img_size32, 3))
images64 = np.empty(len(onlyfiles), dtype=object)
faces64 = np.empty((len(onlyfiles), 224, 224, 3))
for n in range(0, len(onlyfiles)):
   # images32[n] = cv2.imread( join(mypath,onlyfiles[n]))
    #images32[n] = cv2.cvtColor(images32[n],cv2.COLOR_BGR2RGB)
    #faces32[n, : , : , :] = cv2.resize(images32[n][35:-35,35:-35,:], (img_size32, img_size32))
    image = face_recognition.load_image_file(join(mypath,onlyfiles[n]))
    face_locations = face_recognition.face_locations(image)[0]
    #print(face_locations)
    #print(face_locations[0],face_locations[2])
    #print(face_locations[3],face_locations[1])
    #print(image.shape)
    images64[n] = image[face_locations[0]:face_locations[2],face_locations[3]:face_locations[1],:]
    faces64[n, : , : , :] = cv2.resize(images64[n], (224, 224))
    
#results32 = model32.predict(faces32)
results64 = model64.predict(faces64)
#predicted_genders32 = results32[0]
#predicted_genders64 = results64[0]
ages = np.arange(0, 101).reshape(101, 1)
#predicted_ages32 = results32[1].dot(ages).flatten()
predicted_ages64 = results64.dot(ages).flatten()

for n in range(len(onlyfiles)):
    plt.figure()
   # plt.imshow(cv2.resize(images32[n], (img_size64, img_size64)))
    #print("Age " +onlyfiles[n]+ " (32x32): ", int(predicted_ages32[n]))
    #print("Gender " +onlyfiles[n]+ " (32x32): ", "F" if predicted_genders32[n][0] > 0.5 else "M")
    plt.figure()
    plt.imshow(images64[n])
    print("Age " +onlyfiles[n]+ " (64x64): ", predicted_ages64[n])
    #print("Gender " +onlyfiles[n]+ " (64x64): ", "F" if predicted_genders64[n][0] > 0.5 else "M")

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

def generate_data_generator(generator):
    for x_batch,y_batch in generator:
            ages = np.arange(0, 101).reshape(101, 1)
            round_age = [int(pred.dot(ages).flatten()) for pred in y_batch]
            yield x_batch, [y_batch, np.array(round_age)]

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=20,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2)

val_generator = train_datagen.flow_from_directory(
    '../../dataset/wiki_crop/new_database/',
    target_size=(img_size, img_size),
    class_mode='categorical',
    subset='validation',
    shuffle=True)

In [ ]:

def x_difference(val_generator, X):
    print(len(val_generator.classes))
    predictions = model64.predict_generator(generate_data_generator(val_generator),steps=int(6769 / 16))
    print("Done Prediction...")
    ages = np.arange(0, 101).reshape(101, 1)
    predicted_age = predictions[0].dot(ages).flatten()
    predicted_age_regressed = predictions[1]
    
    counter = 0
    counter2 = 0
    counter3 = 0
    #print(predicted_age_regressed)
    
    avg = [(predicted_age[i] + predicted_age_regressed[i])/2 for i in range(len(val_generator.classes))]
    
    #print(avg)
    summ = 0
    for i in range(len(val_generator.classes)):
        real_label = val_generator.classes[i]
        #print(real_label)
        #print(avg[i])
        if abs(real_label - predicted_age[i]) <= X:
            counter += 1
        if abs(real_label - avg[i]) <= X:
            counter2 += 1
        if abs(real_label - predicted_age_regressed[i]) <= X:
            counter3 += 1
        
        summ += abs(real_label - avg[i])
    #return counter / len(val_generator.classes),counter2 / len(val_generator.classes), counter3 / len(val_generator.classes)
    return summ / len(val_generator.classes)
c1 = x_difference(val_generator,10)

print(c1)
#print(c2)
#print(c3)


In [ ]:
model64.save_weights(os.path.join("models", "Tentativa.h5"), overwrite=True)

In [4]:
from os import listdir
from os.path import isfile, join

nb_class = 101

path_dataset = "../../dataset/FGNET/images/"



In [ ]:
images = [ f for f in listdir(path_dataset) if isfile(join(path_dataset,f)) ]

images = sorted(images)

x_batch = [cv2.resize(cv2.cvtColor(cv2.imread(path_dataset+img),cv2.COLOR_BGR2RGB),(224,224)) for img in images]

y_true = [int(name.split('A')[1][:2]) for name in images]

In [ ]:
import random

c = list(zip(x_batch, y_true))

random.shuffle(c)

x_batch, y_true = zip(*c)

x_batch = np.array(x_batch)


percentage = 0.2
x_data = (x_batch[:int(len(x_batch)*(1-percentage))],x_batch[int(len(x_batch)*(1-percentage)):])
y_data = (y_true[:int(len(y_true)*(1-percentage))],y_true[int(len(y_true)*(1-percentage)):])

x_train, y_train = x_data[0], np.array(y_data[0])
x_val, y_val = x_data[1], np.array(y_data[1])

print(x_batch.shape)
print(y_val[2])

In [ ]:
#train and val

y_train2 = to_categorical(y_train, nb_class).astype('float32')
y_val2 = to_categorical(y_val, nb_class).astype('float32')

y_train = y_train.reshape(len(y_train),1)
y_val = y_val.reshape(len(y_val),1)

y_train = np.array(y_train).reshape(-1)
y_val = np.array(y_val).reshape(-1)
#print(y_train2.shape)
#print(x_val.shape)

print(y_train2.shape)

In [9]:
images = [ f for f in listdir(path_dataset) if isfile(join(path_dataset,f)) ]

images = sorted(images)

num_val = int(len(images)*0.2)

train_names = images[num_val:]
val_names = images[:num_val]

print("Size of Training data: ", len(train_names))
print("Size of Validation data: ", len(val_names))

batch_size = 32

def get_batch(list_filenames):
    while True:
        for batches in range(len(list_filenames) // batch_size):
            x_batch = []
            real_age = []
            categorical_age = []
            for filename in list_filenames[batches*batch_size:min(len(list_filenames),(batches+1)*batch_size)]:
            #get real age
                #age = int(filename.split('A')[1][:-4])
                age = int(filename.split('A')[1][:2])
                real_age.append(age)
                #categorical
                cat_age = to_categorical(age,nb_class)
                categorical_age.append(cat_age)
                #get image, color change and resize -> not needed, organize_db does this
                image = cv2.imread(path_dataset+filename)
                if image.shape != (224,224,3):
                    image = cv2.resize(image,(224,224))
                x_batch.append(image)
                #print(image.shape)
            x_batch = np.array(x_batch)
            real_age = np.array(real_age)
            categorical_age = np.array(categorical_age)
            #print(x_batch.shape)
            #print(real_age.shape)
            #print(categorical_age.shape)
            yield x_batch,[categorical_age,real_age]

Size of Training data:  4492
Size of Validation data:  1122


In [18]:
###
import keras
from MyModel import MyModel
from keras.optimizers import SGD, Adam

model = MyModel(224,nb_class=nb_class,trainable=False)()
sgd = Adam(lr=0.0001)

#getBatches
os.environ["CUDA_VISIBLE_DEVICES"]="1"

tbCallBack1 = keras.callbacks.TensorBoard(log_dir='../../dataset/graphs/Graph1', histogram_freq=0, write_graph=True, write_images=True)
tbCallBack2 = keras.callbacks.TensorBoard(log_dir='../../dataset/graphs/Graph2', histogram_freq=0, write_graph=True, write_images=True)
tbCallBack3 = keras.callbacks.TensorBoard(log_dir='../../dataset/graphs/Graph3', histogram_freq=0, write_graph=True, write_images=True)


model.compile(optimizer=sgd, loss=["categorical_crossentropy","MSE"], loss_weights=[0.5,1],
              metrics=['accuracy','MAE'])

print("Start Fitting...")

model.fit_generator(get_batch(train_names),
                    steps_per_epoch=len(train_names) // batch_size, epochs=25, validation_data = get_batch(val_names),validation_steps=len(val_names)// batch_size, callbacks=[tbCallBack1])

sgd = Adam(lr=0.00001)
model.compile(optimizer=sgd, loss=["categorical_crossentropy","MSE"], loss_weights=[0.5,1],
              metrics=['accuracy'])
model.fit_generator(get_batch(train_names),
                    steps_per_epoch=len(train_names) // batch_size, epochs=5, validation_data = get_batch(val_names),validation_steps=len(val_names)// batch_size, callbacks=[tbCallBack2])
sgd = Adam(lr=0.000001)
model.compile(optimizer=sgd, loss=["categorical_crossentropy","MSE"], loss_weights=[0.5,1],
              metrics=['accuracy'])
model.fit_generator(get_batch(train_names),
                    steps_per_epoch=len(train_names) // batch_size, epochs=5, validation_data = get_batch(val_names),validation_steps=len(val_names)// batch_size, callbacks=[tbCallBack3])


Start Fitting...
Epoch 1/25
140/140 [==============================] - 17s 118ms/step - loss: 186.8788 - pred_age_loss: 13.8343 - regress_age_loss: 173.7368 - pred_age_acc: 0.0654 - pred_age_mean_absolute_error: 0.0185 - regress_age_acc: 0.0350 - regress_age_mean_absolute_error: 9.7270 - val_loss: 118.4857 - val_pred_age_loss: 11.2198 - val_regress_age_loss: 106.8188 - val_pred_age_acc: 0.1366 - val_pred_age_mean_absolute_error: 0.0171 - val_regress_age_acc: 0.0420 - val_regress_age_mean_absolute_error: 7.4007
Epoch 2/25
140/140 [==============================] - 17s 122ms/step - loss: 119.2059 - pred_age_loss: 11.1197 - regress_age_loss: 107.6596 - pred_age_acc: 0.1337 - pred_age_mean_absolute_error: 0.0172 - regress_age_acc: 0.0498 - regress_age_mean_absolute_error: 7.4795 - val_loss: 82.0171 - val_pred_age_loss: 5.6671 - val_regress_age_loss: 73.2529 - val_pred_age_acc: 0.3393 - val_pred_age_mean_absolute_error: 0.0134 - val_regress_age_acc: 0.0643 - val_regress_age_mean_absolute_er

140/140 [==============================] - 16s 116ms/step - loss: 48.3085 - pred_age_loss: 0.2303 - regress_age_loss: 43.1941 - pred_age_acc: 0.9350 - pred_age_mean_absolute_error: 0.0025 - regress_age_acc: 0.0804 - regress_age_mean_absolute_error: 4.6602 - val_loss: 26.2160 - val_pred_age_loss: 0.0017 - val_regress_age_loss: 21.2537 - val_pred_age_acc: 1.0000 - val_pred_age_mean_absolute_error: 3.3757e-05 - val_regress_age_acc: 0.1161 - val_regress_age_mean_absolute_error: 3.3839
Epoch 18/25
140/140 [==============================] - 18s 125ms/step - loss: 46.5634 - pred_age_loss: 0.1943 - regress_age_loss: 41.5432 - pred_age_acc: 0.9455 - pred_age_mean_absolute_error: 0.0022 - regress_age_acc: 0.0897 - regress_age_mean_absolute_error: 4.6191 - val_loss: 25.4646 - val_pred_age_loss: 0.0017 - val_regress_age_loss: 20.5798 - val_pred_age_acc: 1.0000 - val_pred_age_mean_absolute_error: 3.3225e-05 - val_regress_age_acc: 0.1205 - val_regress_age_mean_absolute_error: 3.3226
Epoch 19/25
140/

In [ ]:
from keras.models import load_model
import os

os.environ["CUDA_VISIBLE_DEVICES"]="1"


model = load_model('../../dataset/models/state_meh.h5')

In [19]:
from os import listdir
from os.path import isfile, join

path_test = "../../dataset/FGNET_TEST/"

images = [ f for f in listdir(path_test) if isfile(join(path_test,f)) ]

images = sorted(images)

x_test = [cv2.resize(cv2.cvtColor(cv2.imread(path_test+img),cv2.COLOR_BGR2RGB),(224,224)) for img in images]

y_test = [int(name.split('A')[1][:2]) for name in images]

y_pred = model.predict(np.array(x_test))

#for img in x_test[:10]:
#    plt.figure()
#    plt.imshow(img)

In [20]:
ages = np.arange(0, nb_class).reshape(nb_class, 1)
predicted_age = y_pred[0].dot(ages).flatten()
predicted_age_regressed = y_pred[1]

avg = [(predicted_age[i] + predicted_age_regressed[i])/2 for i in range(len(x_test))]

In [21]:
summ1 = 0
summ2 = 0
summ3 = 0
for i in range(len(x_test)):
    summ1 += abs(y_test[i] - avg[i])
    summ2 += abs(y_test[i] - predicted_age[i])
    summ3 += abs(y_test[i] - predicted_age_regressed[i])

print(summ1/len(x_test), " -- AVG")
print(summ2/len(x_test), " -- CLASS")
print(summ3/len(x_test), " -- REG")

[3.4268868]  -- AVG
3.600441425922433  -- CLASS
[4.509377]  -- REG


In [ ]:
model.save('../../dataset/models/state_art_maybe.h5')

In [ ]:
import dlib
import face_recognition

detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor("predictor/shape_predictor_5_face_landmarks.dat")

mypath='test/'
onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
#images32 = np.empty(len(onlyfiles), dtype=object)
#faces32 = np.empty((len(onlyfiles), img_size32, img_size32, 3))
images64 = np.empty(len(onlyfiles), dtype=object)
faces64 = np.empty((len(onlyfiles), 224, 224, 3))
for n in range(0, len(onlyfiles)):
   # images32[n] = cv2.imread( join(mypath,onlyfiles[n]))
    #images32[n] = cv2.cvtColor(images32[n],cv2.COLOR_BGR2RGB)
    #faces32[n, : , : , :] = cv2.resize(images32[n][35:-35,35:-35,:], (img_size32, img_size32))
    image = face_recognition.load_image_file(join(mypath,onlyfiles[n]))
    
    dets = detector(image, 1)
    
    if len(dets) == 0:
        continue
    
    faces = dlib.full_object_detections()
    for detection in dets:
        faces.append(sp(image, detection))
    
    images = dlib.get_face_chips(image, faces, size=320, padding=0.5)
    
    images64[n] = images[0]
    
    
    
    #print(face_locations)
    #print(face_locations[0],face_locations[2])
    #print(face_locations[3],face_locations[1])
    #print(image.shape)
    #images64[n] = image[face_locations[0]:face_locations[2],face_locations[3]:face_locations[1],:]
    faces64[n, : , : , :] = cv2.resize(images64[n], (224, 224))
    
#results32 = model32.predict(faces32)
y_pred = model.predict(faces64)

ages = np.arange(0, nb_class).reshape(nb_class, 1)
predicted_age = y_pred[0].dot(ages).flatten()
predicted_age_regressed = y_pred[1]

avg = [(predicted_age[i] + predicted_age_regressed[i])/2 for i in range(len(onlyfiles))]

for n in range(len(onlyfiles)):
    plt.figure()
   # plt.imshow(cv2.resize(images32[n], (img_size64, img_size64)))
    #print("Age " +onlyfiles[n]+ " (32x32): ", int(predicted_ages32[n]))
    #print("Gender " +onlyfiles[n]+ " (32x32): ", "F" if predicted_genders32[n][0] > 0.5 else "M")
    plt.figure()
    plt.imshow(images64[n])
    print("AVG " +onlyfiles[n]+ " (64x64): ", avg[n])
    print("Class " +onlyfiles[n]+ " (64x64): ", predicted_age[n])
    print("Regress " +onlyfiles[n]+ " (64x64): ", predicted_age_regressed[n])
    #print("Gender " +onlyfiles[n]+ " (64x64): ", "F" if predicted_genders64[n][0] > 0.5 else "M")